In [38]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
from torchtext.vocab import vocab
from torchtext.data import get_tokenizer
from collections import OrderedDict
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(0) # Set for testing purposes, please do not change!

In [39]:
def get_vocab(corpora):
    tokens = set()
    tokenizer = get_tokenizer(None)
    for line in corpora:
        words = tokenizer(line)
        for word in words:
            for token in word:
                tokens.add(token)
    v1 = vocab(OrderedDict([(token, 1) for token in tokens]), specials=["<end>", "<pad>"])
    return v1

In [40]:
corpora = """Perda de pontos no futebol por racismo de torcedores ainda é realidade distante
Brasil é o único país entre as principais ligas a prever a punição esportiva, que ainda não foi efetivamente aplicada
Compartilhar nas redes sociais "Tostão: Existe uma inversão no aprendizado esportivo"
Crianças precisam se divertir com a bola antes de aprender técnica e regras
Compartilhar nas redes sociais "Financial Times: Netflix fecha acordo de R$ 24,7 bilhões para transmitir luta livre nos EUA"
Netflix fecha acordo de R$ 24,7 bilhões para transmitir luta livre nos EUA
Serviço de streaming exibirá a atração a partir de 2025 como parte de um acordo de dez anos
Compartilhar nas redes sociais "Futebol Internacional: Udinese fará jogo com portões fechados como punição por insultos a Maignan"
Udinese fará jogo com portões fechados como punição por insultos a Maignan
Goleiro francês Mike Maignan, do Milan, foi alvo de insultos por parte de torcedores do time de Udine
Compartilhar nas redes sociais "Esporte: Corinthians e Cruzeiro disputam título da Copinha na Neo Química Arena"
Corinthians e Cruzeiro disputam título da Copinha na Neo Química Arena
Partida decisiva acontece nesta quinta-feira (25), aniversário de São Paulo
Compartilhar nas redes sociais "Como É Que É?: Bets: por que apostas online viraram um fenômeno no Brasil?"
Bets: por que apostas online viraram um fenômeno no Brasil?
Isabella Faria conversa com João Gabriel às 18h, ao vivo
Compartilhar nas redes sociais "O Mundo É uma Bola: Os campeões da preferência e da resistência"
Bruno Fernandes, do Manchester United, esteve em campo 76 vezes em 2023, cinco a mais que Gustavo Gómez, do Palmeiras, e Rodrygo, do Real Madrid
Compartilhar nas redes sociais "Esporte: Gigi Riva, lenda do futebol italiano, morre aos 79 anos"
Gigi Riva, lenda do futebol italiano, morre aos 79 anos
Jogador foi campeão europeu em 1968 e vice-campeão mundial em 1970
Compartilhar nas redes sociais "Sandro Macedo: O efervescente mercadão de janeiro no futebol"
Janela de transferências dá a impressão de que clubes brasilianos são mais opulentos do que realmente são
Compartilhar nas redes sociais "Esporte: Equipe brasileira de curling vence pela primeira vez nas Olimpíadas de Inverno"
Equipe brasileira de curling vence pela primeira vez nas Olimpíadas de Inverno
Modalidade é conhecida pelo uso de uma espécie de vassoura para deixar o gelo mais liso
Compartilhar nas redes sociais "Futebol Internacional: Presidente da Fifa defende derrota automática em casos de racismo nos estádios"
Presidente da Fifa defende derrota automática em casos de racismo nos estádios
Manifestações contra jogadores negros ocorreram na Itália e na Inglaterra no sábado
Compartilhar nas redes sociais "Café da Manhã: Podcast discute a regulamentação e o impacto das apostas esportivas online"
Podcast discute a regulamentação e o impacto das apostas esportivas online
Bets chegam a 15% da população, segundo o Datafolha; governo mira arrecadação, e especialistas alertam para riscos
Compartilhar nas redes sociais "Campeonato Paulista: Romero marca de voleio, e Corinthians estreia com vitória no Paulista"
Romero marca de voleio, e Corinthians estreia com vitória no Paulista"""

In [41]:
v1 = get_vocab(corpora)

In [42]:
class Encoder(nn.Module):
    def __init__(self, char_num_embeddings=10, char_embedding_dim=5, word_embedding_dim=2, word_chars_size=5):
        super(Encoder, self).__init__()
        self.char_num_embeddings = char_num_embeddings
        self.char_embedding_dim = char_embedding_dim
        self.word_embedding_dim = word_embedding_dim
        self.word_chars_size = word_chars_size
        
        self.embedding = nn.Sequential(
            nn.Embedding(char_num_embeddings, char_embedding_dim, max_norm=True)
        )
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=(3, char_embedding_dim), padding=(2, 0)),
            nn.ReLU(True),
            nn.Conv2d(8, 16, kernel_size=(3, 1), padding=(2, 0)),
            nn.Flatten(1, 3),
            nn.Linear((self.word_chars_size + 4) * 16, word_embedding_dim)
        )
    def forward(self, x):
        x = x.view(len(x), 1, self.word_chars_size)
        return self.encoder(self.embedding(x))
    
class Decoder(nn.Module):
    def __init__(self, char_num_embeddings=10, char_embedding_dim=5, word_embedding_dim=2, word_chars_size=5):
        super(Decoder, self).__init__()
        self.char_num_embeddings = char_num_embeddings
        self.char_embedding_dim = char_embedding_dim
        self.word_embedding_dim = word_embedding_dim
        self.word_chars_size = word_chars_size
        
        self.decoder = nn.Sequential(
            nn.Linear(word_embedding_dim, (self.word_chars_size + 4) * 16),
            nn.Unflatten(1, (16, (self.word_chars_size + 4), 1)),
            nn.ConvTranspose2d(16, (self.word_chars_size + 4), kernel_size=(1, 1)),
            nn.ReLU(True),
            nn.ConvTranspose2d((self.word_chars_size + 4), 1, kernel_size=(1, char_embedding_dim), padding=(2, 0)),
        )
    def forward(self, x):
        return self.decoder(x)

In [102]:
class CBOW(nn.Module):
    def __init__(self, encoder, context_size=2, hidden_dim=16):
        super(CBOW, self).__init__()
        self.context_size = context_size
        self.encoder = encoder
        
        self.cbow = nn.Sequential(
            nn.Linear(context_size * self.encoder.word_embedding_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, self.encoder.char_num_embeddings * self.encoder.word_chars_size),
        )
    def forward(self, x):
        size = len(x)
        x = x.view(size * self.context_size, self.encoder.word_chars_size)
        x = self.encoder(x)
        x = x.view(size, self.context_size * self.encoder.word_embedding_dim)
        x = self.cbow(x)
        x = x.view(size, self.encoder.word_chars_size, self.encoder.char_num_embeddings)
        x = F.log_softmax(x, dim=2)
        return x

In [103]:
device = 'cpu'
lr = 0.00001
char_num_embeddings = len(v1)  # size of the dictionary of embeddings
char_embedding_dim = 8  # dimension of the char embedding
word_embedding_dim = 2  # dimension of the word embedding
word_chars_size = 6  # size of chars in a word
context_size = 3  # size of words to use as context in CBOW

encoder = Encoder(
    char_num_embeddings=char_num_embeddings,
    char_embedding_dim=char_embedding_dim,
    word_embedding_dim=word_embedding_dim,
    word_chars_size=word_chars_size,
    ).to(device)
encoder_opt = torch.optim.Adam(encoder.parameters(), lr=lr)

decoder = Decoder(
    char_num_embeddings=char_num_embeddings,
    char_embedding_dim=char_embedding_dim,
    word_embedding_dim=word_embedding_dim,
    word_chars_size=word_chars_size,
    ).to(device)
decoder_opt = torch.optim.Adam(decoder.parameters(), lr=lr)

cbow = CBOW(encoder, context_size=context_size)

In [104]:
encoder(torch.LongTensor([[1, 2, 4, 5, 9, 1], [1, 2, 4, 5, 10, 1], [1, 2, 4, 5, 10, 1]])).size()

torch.Size([3, 2])

In [105]:
decoder(encoder(torch.LongTensor([[1, 2, 4, 5, 9, 1], [1, 2, 4, 5, 10, 1], [1, 2, 4, 5, 10, 1]])))

tensor([[[[ 0.5152,  0.1101,  0.4665,  0.4608,  0.1533,  0.2767,  0.3929,
           -0.0191],
          [ 0.4220,  0.1719,  0.3538,  0.3417,  0.3096,  0.2442,  0.1459,
            0.0635],
          [ 0.3111,  0.1460,  0.3295,  0.3408,  0.1928,  0.2531,  0.3139,
            0.1144],
          [ 0.2529,  0.3060,  0.2864,  0.4245,  0.1493,  0.2880,  0.3163,
            0.2132],
          [ 0.5836,  0.2938,  0.3946,  0.3397,  0.1577,  0.2219,  0.3330,
            0.2162],
          [ 0.3986,  0.1081,  0.3163,  0.3311,  0.1863,  0.2213,  0.2560,
            0.0717]]],


        [[[ 0.5179,  0.1070,  0.4661,  0.4555,  0.1559,  0.2735,  0.3882,
           -0.0240],
          [ 0.4167,  0.1704,  0.3509,  0.3439,  0.3050,  0.2455,  0.1493,
            0.0658],
          [ 0.3098,  0.1482,  0.3265,  0.3342,  0.1948,  0.2506,  0.3094,
            0.1162],
          [ 0.2455,  0.3063,  0.2836,  0.4191,  0.1477,  0.2869,  0.3182,
            0.2140],
          [ 0.5812,  0.2926,  0.3945,  0.3423,

In [100]:
cbow(torch.LongTensor([[[1, 2, 4, 5, 9, 1], [1, 2, 4, 5, 10, 1], [1, 2, 4, 5, 10, 1]], [[1, 2, 4, 5, 11, 1], [1, 2, 4, 5, 10, 1], [1, 2, 3, 5, 12, 1]]]))

tensor([[[-4.5013, -4.3151, -4.3870, -4.0942, -4.7874, -3.9470, -3.6737,
          -4.8199, -4.8802, -4.4568, -3.8225, -4.4814, -3.6739, -4.2833,
          -4.6093, -4.1806, -4.0067, -4.1466, -4.0712, -4.5833, -4.4849,
          -4.9377, -3.8950, -3.9672, -4.3191, -4.6407, -4.5979, -4.7272,
          -4.5984, -5.2834, -4.6484, -3.9636, -4.2710, -5.1401, -4.7937,
          -3.8213, -4.4092, -4.3184, -4.9511, -4.6120, -4.2943, -3.6914,
          -4.6399, -4.2426, -4.2834, -4.4456, -4.5493, -4.3814, -4.1899,
          -4.3501, -5.6203, -4.1577, -3.8655, -4.5871, -4.6649, -4.5750,
          -4.9073, -4.2124, -3.9721, -3.9324, -4.4779, -4.3933, -4.4485,
          -4.3123, -4.6212, -4.6818, -4.1329, -3.9506, -4.0723, -4.9480,
          -4.3912, -4.3138, -4.5372, -4.9883, -4.4857, -4.8501, -4.4514],
         [-4.4666, -4.6144, -4.0790, -4.5302, -4.3440, -3.8733, -4.4915,
          -4.2243, -4.3705, -4.3973, -4.8454, -4.5123, -4.6616, -4.5447,
          -4.9126, -4.6122, -4.5586, -4.3699, -4.5

In [101]:
import torch.nn.functional as F
import torch.optim as optim

CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        raise Exception(context_idxs)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


Exception: tensor([80, 43])

In [29]:
model(torch.tensor([18, 28, 45]))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30 and 20x128)

In [217]:
torch.tensor([word_to_ix[target]])

tensor([96])